# This is a secondary exploration notebook to further evaluate models and predictions.
# Not to be taken as the main one, feel free to play with it.

In [8]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from PIL import Image, ImageEnhance
import random
from tqdm.auto import tqdm
import gdown
import shutil
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import yaml
try:
    with open ("../config.yaml", 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print('Error reading the config file')
import sys
import pickle

# ML Models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import load_img

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

## Loading models, prediction dictionary and pictures to evaluate.

In [9]:
bts = tf.keras.models.load_model(config['models']+'bts.model')
trl = tf.keras.models.load_model(config['models']+'trl.model')

## Renaming models for easier read

In [100]:
bts._name = 'BTS'

In [101]:
bts.name

'BTS'

In [102]:
trl._name = 'TRL'

In [103]:
trl.name

'TRL'

In [10]:
with open('../src/lib/pred_dict.pickle', 'rb') as handle:
    pred_dict = pickle.load(handle)

In [69]:
glioma1 = config['diagnose']+'glioma1.jpg'
glioma2 = config['diagnose']+'glioma2.jpg'
glioma3 = config['diagnose']+'glioma3.jpg'
glioma4 = config['diagnose']+'glioma4.jpg'
glioma5 = config['diagnose']+'glioma5.jpg'

meningioma1 = config['diagnose']+'meningioma1.jpg'
meningioma2 = config['diagnose']+'meningioma2.jpg'
meningioma3 = config['diagnose']+'meningioma3.jpg'
meningioma4 = config['diagnose']+'meningioma4.jpg'

notumor1 = config['diagnose']+'notumor1.jpg'
notumor2 = config['diagnose']+'notumor2.jpg'
notumor3 = config['diagnose']+'notumor3.jpg'

pituitary1 = config['diagnose']+'pituitary1.jpg'
pituitary2 = config['diagnose']+'pituitary2.jpg'

## Functions version 1 - with second opinion from model TRL when no tumor is detected.
## We take advantage of TRL model performing better at detecting no tumors, so if it detects a tumor after the BTS model didn't, it will recommend user to consult a doctor for further revision.

In [17]:
def image_prep(image):
    img = Image.open(image)
    img = Image.fromarray(np.uint8(img))
    img = ImageEnhance.Brightness(img).enhance(1)
    img = ImageEnhance.Contrast(img).enhance(1)
    return img

def conclusion(number):
    if number == 0:
        return pred_dict[0]
    elif number == 1:
        return pred_dict[1]
    elif number == 2:
        return pred_dict[2]
    elif number == 3:
        return pred_dict[3]
    else:
        return 'Sorry, not clear'

def diagnose(image, model=bts):    
    img = image_prep(image)
    x = np.array(img.resize((128,128)))
    x = x.reshape(1,128,128,3)
    x = np.array(x)/255.0
    res = model.predict_on_batch(x)
    classification = np.where(res == np.amax(res))[1][0]
    diagnose = (conclusion(classification))
    if classification == 1:
        if model == bts:
            print('BTS model has concluded that there is no tumor.')
            print('Using TRL model to make sure it is not a false negative, as this model performs better in these cases.')
            res = trl.predict_on_batch(x)
            classification = np.where(res == np.amax(res))[1][0]
            if classification == 1:
                diagnose = (conclusion(classification))
                image = img.resize((500,500))
                print('TRL model has confirmed the diagnosis:')
                print('No tumor detected')
                imshow(image)
            else:
                diagnose = (conclusion(classification))
                image = img.resize((500,500))
                print('TLR model has concluded:'+diagnose)
                print(diagnose)
                print('Please consult a doctor as there is no clear conclusion.')
                imshow(image)

    else:
        image = img.resize((500,500))
        print(diagnose)
        imshow(image)

## Version 2 function: displays the confidence levels.

In [105]:
def image_prep(image):
    img = Image.open(image)
    img = Image.fromarray(np.uint8(img))
    img = ImageEnhance.Brightness(img).enhance(1)
    img = ImageEnhance.Contrast(img).enhance(1)
    return img

def diagnose(image, model=bts):    
    img = image_prep(image)
    x = np.array(img.resize((128,128)))
    x = x.reshape(1,128,128,3)
    x = np.array(x)/255.0
    res = model.predict_on_batch(x)
    score = max(res[0])
    lbl = tf.nn.softmax(res[0])
    print("{}. confidence: {:.2f} %. Model used: {}"
    .format(pred_dict[np.argmax(lbl)], 100 * score, model.name))

In [106]:
diagnose(glioma1)
diagnose(glioma1, trl)
diagnose(glioma2)
diagnose(glioma2, trl)
diagnose(glioma3)
diagnose(glioma3, trl)
diagnose(glioma4)
diagnose(glioma4, trl)

Tumor detected - Glioma. confidence: 99.88 %. Model used: BTS
Tumor detected - Glioma. confidence: 100.00 %. Model used: TRL
Tumor detected - Glioma. confidence: 100.00 %. Model used: BTS
Tumor detected - Glioma. confidence: 100.00 %. Model used: TRL
Tumor detected - Glioma. confidence: 100.00 %. Model used: BTS
Tumor detected - Glioma. confidence: 100.00 %. Model used: TRL
Tumor detected - Glioma. confidence: 98.12 %. Model used: BTS
Tumor detected - Glioma. confidence: 100.00 %. Model used: TRL


In [104]:
diagnose(notumor1)
diagnose(notumor1, trl)
diagnose(notumor2)
diagnose(notumor2, trl)
diagnose(notumor3)
diagnose(notumor3, trl)

No tumor detected. confidence: 99.63 %. Model used: BTS
No tumor detected. confidence: 100.00 %. Model used: TRL
No tumor detected. confidence: 100.00 %. Model used: BTS
No tumor detected. confidence: 99.82 %. Model used: TRL
No tumor detected. confidence: 100.00 %. Model used: BTS
No tumor detected. confidence: 100.00 %. Model used: TRL


In [70]:
diagnose(meningioma1)
diagnose(meningioma1, trl)
diagnose(meningioma2)
diagnose(meningioma2, trl)
diagnose(meningioma3)
diagnose(meningioma3, trl)
diagnose(meningioma4)
diagnose(meningioma4, trl)

Tumor detected - Meningioma. Confidence: 98.46 %.
Tumor detected - Meningioma. Confidence: 99.99 %.
Tumor detected - Meningioma. Confidence: 100.00 %.
Tumor detected - Meningioma. Confidence: 100.00 %.
Tumor detected - Meningioma. Confidence: 100.00 %.
Tumor detected - Meningioma. Confidence: 100.00 %.
Tumor detected - Meningioma. Confidence: 96.16 %.
Tumor detected - Meningioma. Confidence: 99.01 %.


In [97]:
diagnose(pituitary1)
diagnose(pituitary1, trl)
diagnose(pituitary2)
diagnose(pituitary2, trl)

Tumor detected - Pituitary. confidence: 100.00 %. Model used: Brain_Tumor_Scanner
Tumor detected - Pituitary. confidence: 100.00 %. Model used: Brain_Tumor_Scanner_TRL
Tumor detected - Pituitary. confidence: 100.00 %. Model used: Brain_Tumor_Scanner
Tumor detected - Pituitary. confidence: 100.00 %. Model used: Brain_Tumor_Scanner_TRL
